# Revisão do Fluxo de Trabalho de Machine Learning

Trabalharemos com o conjunto de dados de carros.

Nosso objetivo é prever o preço dos carros.

Começaremos carregando os dados com a biblioteca **pandas**.

In [2]:
import pandas as pd

data = pd.read_csv('dados/car-sales-extended-missing-data.csv')

In [3]:
data.head(15)

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
5,Honda,Red,42652.0,4.0,23883.0
6,Toyota,Blue,163453.0,4.0,8473.0
7,Honda,White,NaN,4.0,20306.0
8,NaN,White,130538.0,4.0,9374.0
9,Honda,Blue,51029.0,4.0,26683.0


Vejamos os tipos de dados de nossas features.

In [4]:
data.dtypes

Make              object
Colour            object
Odometer (KM)    float64
Doors            float64
Price            float64
dtype: object

Veja que temos strings em nosso conjunto de dados.

Observe também que temos dados faltantes em nosso conjunto de dados.

Pandas nos permite saber quantos dados estão faltando.

In [5]:
data.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

Precisamos deixar os dados prontos para o algoritmo de Machine Learning.

Para isso, vamos seguir algumas etapas:

1. Preencher e/ou eliminar os dados faltantes
2. Converter strings para números
3. Construir o Modelo com os dados

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

# Eliminamos as amostras da coluna Price que estão faltando
data.dropna(subset=['Price'], inplace=True)

# Definimos diferentes features e transformers pipelines
categorical_features = ['Make','Colour']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

door_feature = ['Doors']
door_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=4))
])

numeric_features = ['Odometer (KM)']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

# Configuramos as etapas de pré-processamento (preencher valores faltantes e então converter strings para números)
preprocessor = ColumnTransformer(transformers=[
    ('cat', categorical_transformer, categorical_features),
    ('door', door_transformer, door_feature),
    ('num', numeric_transformer, numeric_features)
])

# Criamos uma pipeline de pré-processamento e modelagem
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
])

# Dividimos os dados
X = data.drop('Price', axis=1)
y = data['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Ajustamos os dados e vimos o seu desempenho
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.20222402675354056

Também é possível usar **GridSearchCV** ou **RandomizedSearchCV** em nossa Pipeline.

Usaremos **GridSearchCV** em nossa Pipeline de Regressão.

In [9]:
pipe_grid = {
    "preprocessor__num__imputer__strategy": ["mean", "median"],
    "model__n_estimators": [100, 1000],
    "model__max_features": ["auto"],
    "model__min_samples_split": [2, 4]
}

gs_model = GridSearchCV(model, pipe_grid, cv=5, verbose=2)
gs_model.fit(X_train, y_train);

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean, total=   0.5s
[CV] model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean, total=   0.4s
[CV] model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean 
[CV]  model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean, total=   0.4s
[CV] model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean 
[CV]  model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean, total=   0.6s
[CV] model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean 
[CV]  model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean, total=   0.4s
[CV] model__max_features=auto, model__min_samples_s

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  1.2min finished


E agora podemos avaliar o modelo encontrado com **GridSearchCV**.

In [10]:
gs_model.score(X_test, y_test)

0.24748074433508663

Conseguimos aprimorar o nosso modelo ao encontrarmos os hiperparâmetros mais otimizados.

Apesar disso, temos uma limitação de features nesse conjunto de dados.